In [ ]:
import json
from openai import OpenAI
import sys
sys.path.append('../')
from llm_literature import Toolbox, Entity
import voyageai

def process_user_query(user_query, stored_field="medical_research_using_LLM"):
    """
    Processes a user query using OpenAI's chat completion API and custom tools.

    Args:
        user_query (str): The user's query.
        stored_field (str, optional): The stored field of research. Defaults to "medical_research_using_LLM".

    Returns:
        str: The final response from the model.
    """

    available_functions = {
        "author_collaboration": Toolbox.author_collaboration,
        "certain_author": Toolbox.certain_author,
        "certain_entity": Toolbox.certain_entity,
    }
    # Initailize tools
    author_collaboration_tool = {
        'type': 'function',
        'function': {
            'name': 'author_collaboration',
            'description': 'Access the information about the collaboration network of authors in certain area.',
            'parameters': {
                'type': 'object',
                'required': ['field'],
                'properties': {
                    'field': {'type': 'string', 'description': 'The field of the discussion topic. Notice that every key point should be included. You should use the words in users initial query as much as possible.'},
                },
            },
        },
    }

    certain_author_tool = {
        'type': 'function',
        'function': {
            'name': 'certain_author',
            'description': 'Access the information about one of the authors in a certain area regarding articles and collaborations.',
            'parameters': {
                'type': 'object',
                'required': ['field', 'name'],
                'properties': {
                    'field': {'type': 'string', 'description': 'The field of the discussion topic. Notice that every key point should be included. You should use the words in users initial query as much as possible.'},
                    'name': {'type': 'string', 'description': 'The name of the author user wants to query.'},
                },
            },
        },
    }

    certain_entity_tool = {
        'type': 'function',
        'function': {
            'name': 'certain_entity',
            'description': 'The information about certain entity or concept',
            'parameters': {
                'type': 'object',
                'required': ['entity_name'],
                'properties': {
                    'entity_name': {'type': 'string', 'description': 'The name of the entity or concept according to the query of the user. You should use the words in users initial query.'},
                },
            },
        },
    }

    messages = [{'role': 'user', 'content': user_query}]


    # Initailize embedding and chat client
    chat_client = OpenAI(api_key="ollama", base_url='http://localhost:11434/v1') # Work both for ollama and openai model
    embedding_client = voyageai.Client(api_key="Your Voyageai API key")
    Entity.set_embedding_client(embedding_client)

    response = chat_client.chat.completions.create(
        model="qwen2.5:7b",  # Change to your model
        messages=messages,
        tools=[author_collaboration_tool, certain_author_tool, certain_entity_tool],
        tool_choice="auto"
    )

    response_message = response.choices[0].message

    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            print(f"Name of the used function: {function_name}")
            function_to_call = available_functions.get(function_name)
            function_args = json.loads(tool_call.function.arguments)

            if function_name == "author_collaboration":
                field = stored_field
                if not stored_field and "field" in function_args:
                    stored_field = function_args.get("field")
                    field = stored_field
                
                tools = Toolbox(field=field)
                function_response = tools.author_collaboration()

            elif function_name == "certain_author":
                field = stored_field
                if not stored_field and "field" in function_args:
                    stored_field = function_args.get("field")
                    field = stored_field
                name = function_args.get("name")
                tools = Toolbox(field=field)
                function_response = tools.certain_author(author=name)

            elif function_name == "certain_entity":
                entity_name = function_args.get("entity_name")
                tools = Toolbox(field=stored_field) # Using the stored field here
                function_response = tools.certain_entity(name=entity_name)

            messages.append(response_message)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )

        second_response = chat_client.chat.completions.create(
            model="qwen2.5:7b",
            messages=messages,
        )
        return second_response.choices[0].message.content
    else:
        print("no tool was used")
        return response_message.content



In [2]:
# Example usage:
user_query = "What is machine learning used in the field of medical research"
response = process_user_query(user_query)
print(response)

Name of the used function: certain_entity
Machine learning is increasingly being used in the field of medical research for a variety of applications. Here are some key areas and specific examples:

1. **MACHINE LEARNING RESEARCH**
   - A resurgence in machine learning research driven by increasing access to health data worldwide, focusing on computationally efficient algorithms for healthcare applications.

2. **MACHINE LEARNING-BASED MULTI-LABEL MEDICAL TEXT CLASSIFICATION**
   - Uses of machine learning techniques to classify medical texts into multiple labels.

3. **ARTIFICIAL INTELLIGENCE IN MEDICINE**
   - A conference that focuses on the application of artificial intelligence in medicine, covering topics such as patient management, ethical considerations, and predictive modeling in healthcare.

4. **CLINICAL NLP RESEARCHERS**
   - Clinical Natural Language Processing (NLP) researchers use NLP techniques to analyze pathology reports and other medical documents.

5. **MEDICAL NLP T